In [1]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ================================

<img src="https://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_models_05-retrieval-model/nvidia_logo.png" style="width: 90px; float: right;">

# Building a Retrieval Model with Merlin Models

This notebook is created using the latest stable [merlin-tensorflow](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/merlin/containers/merlin-tensorflow/tags) container. 

In large scale recommender systems pipelines, the size of the item catalog (number of unique items) might be in the order of millions. At such scale, a typical setup is having two-stage pipeline, where a faster candidate retrieval model quickly extracts thousands of relevant items and a then a more powerful ranking model (i.e. with more features and more powerful architecture) ranks the top-k items that are going to be displayed to the user. For ML-based candidate retrieval model, as it needs to quickly score millions of items for a given user, a popular choices are models that can produce recommendation scores by just computing the dot product the user embeddings and item embeddings. Popular choices of such models are **Matrix Factorization**, which learns low-rank user and item embeddings, and the **Two-Tower architecture**, which is a neural network with two MLP towers where both user and item features are fed to generate user and item embeddings in the output.


### Dataset

In this notebook, we are building a Two-Tower model for Item Retrieval task using synthetic datasets that are mimicking the real [Ali-CCP: Alibaba Click and Conversion Prediction](https://tianchi.aliyun.com/dataset/dataDetail?dataId=408#1) dataset.
### Learning objectives
- Preparing the data with NVTabular
- Training and evaluating Two-Tower model with Merlin Models
- Exporting the model for deployment
- Generating the top K recommendations from the trained model

### Importing Libraries

In [2]:
import os

import nvtabular as nvt
from nvtabular.ops import *
from merlin.models.utils.example_utils import workflow_fit_transform

from merlin.schema.tags import Tags

import merlin.models.tf as mm
from merlin.io.dataset import Dataset
from merlin.models.utils.dataset import unique_rows_by_features

import tensorflow as tf

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2022-11-29 18:57:44.192501: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 18:57:45.381471: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16254 MB memory:  -> device: 0, name: Quadro GV100, pci bus id: 0000:15:00.0, compute capability: 7.0
/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.

In [3]:
# disable INFO and DEBUG logging everywhere
import logging

logging.disable(logging.WARNING)

### Feature Engineering with NVTabular

Let's generate synthetic train and validation dataset objects.

In [4]:
from merlin.datasets.synthetic import generate_data

DATA_FOLDER = os.environ.get("DATA_FOLDER", "/workspace/data/")
NUM_ROWS = os.environ.get("NUM_ROWS", 1000000)
SYNTHETIC_DATA = eval(os.environ.get("SYNTHETIC_DATA", "True"))

if SYNTHETIC_DATA:
    train, valid = generate_data("aliccp-raw", int(NUM_ROWS), set_sizes=(0.7, 0.3))
else:
    train = nvt.Dataset(DATA_FOLDER + "/train/*.parquet")
    valid = nvt.Dataset(DATA_FOLDER + "/valid/*.parquet")

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


In [5]:
train = train.to_ddf().compute()
valid = valid.to_ddf().compute()

We keep only positive interactions where clicks==1 in the dataset.

In [6]:
train = train.loc[train['click']==1].reset_index(drop=True)
valid = valid.loc[valid['click']==1].reset_index(drop=True)

We can drop the target column since in this example we will only use positive interactions and then generate negative samples via negative sampling technique.

In [7]:
train = train.drop(['click', 'conversion'], axis=1)
valid = valid.drop(['click', 'conversion'], axis=1)

Create Dataset objects

In [8]:
train = Dataset(train)
valid = Dataset(valid)

Define output path for the processed parquet files

In [9]:
output_path = os.path.join(DATA_FOLDER, "processed")

In [10]:
category_temp_directory = os.path.join(DATA_FOLDER, "categories")
user_id = ["user_id"] >> Categorify(out_path=category_temp_directory) >> TagAsUserID()
item_id = ["item_id"] >> Categorify(out_path=category_temp_directory) >> TagAsItemID()

item_features = ["item_category", "item_shop", "item_brand"] >> Categorify(out_path=category_temp_directory) >> TagAsItemFeatures()

user_features = (
    [
        "user_shops",
        "user_profile",
        "user_group",
        "user_gender",
        "user_age",
        "user_consumption_2",
        "user_is_occupied",
        "user_geography",
        "user_intentions",
        "user_brands",
        "user_categories",
    ]
    >> Categorify(out_path=category_temp_directory)
    >> TagAsUserFeatures()
)

outputs = user_id + item_id + item_features + user_features

With `transform_aliccp` function, we can execute fit() and transform() on the raw dataset applying the operators defined in the NVTabular workflow pipeline above. The processed parquet files are saved to output_path.

In [11]:
from merlin.datasets.ecommerce import transform_aliccp

transform_aliccp((train, valid), output_path, nvt_workflow=outputs)

## Building a Two-Tower Model with Merlin Models

We will use Two-Tower Model for item retrieval task. Real-world large scale recommender systems have hundreds of millions of items (products) and users. Thus, these systems often composed of two stages: candidate generation (retrieval) and ranking (scoring the retrieved items). At candidate generation step, a subset of relevant items from large item corpus is retrieved. You can read more about two stage Recommender Systems here. In this example, we're going to focus on the retrieval stage.

A Two-Tower Model consists of item (candidate) and user (query) encoder towers. With two towers, the model can learn representations (embeddings) for queries and candidates separately. 

<img src="./images/TwoTower.png"  width="30%">

Image Adapted from: [Off-policy Learning in Two-stage Recommender Systems](https://dl.acm.org/doi/abs/10.1145/3366423.3380130)

We use the `schema` object to define our model.

In [12]:
train = Dataset(os.path.join(output_path, "train", "*.parquet"))
valid = Dataset(os.path.join(output_path, "valid", "*.parquet"))

Select features with user and item tags, and be sure to exclude target column.

In [13]:
schema = train.schema.select_by_tag([Tags.ITEM_ID, Tags.USER_ID, Tags.ITEM, Tags.USER])
train.schema = schema
valid.schema = schema

We can print out the feature column names.

In [14]:
schema

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.start_index,properties.cat_path,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.domain.min,properties.domain.max,properties.domain.name
0,user_id,"(Tags.USER, Tags.USER_ID, Tags.CATEGORICAL, Ta...",int64,False,False,None,0.0,0.0,0.0,/workspace/data/categories/categories/unique.u...,664.0,61.0,0,663,user_id
1,item_id,"(Tags.CATEGORICAL, Tags.ITEM, Tags.ITEM_ID, Ta...",int64,False,False,None,0.0,0.0,0.0,/workspace/data/categories/categories/unique.i...,678.0,62.0,0,677,item_id
2,item_category,"(Tags.CATEGORICAL, Tags.ITEM)",int64,False,False,None,0.0,0.0,0.0,/workspace/data/categories/categories/unique.i...,678.0,62.0,0,677,item_category
3,item_shop,"(Tags.CATEGORICAL, Tags.ITEM)",int64,False,False,None,0.0,0.0,0.0,/workspace/data/categories/categories/unique.i...,678.0,62.0,0,677,item_shop
4,item_brand,"(Tags.CATEGORICAL, Tags.ITEM)",int64,False,False,None,0.0,0.0,0.0,/workspace/data/categories/categories/unique.i...,678.0,62.0,0,677,item_brand
5,user_shops,"(Tags.USER, Tags.CATEGORICAL)",int64,False,False,None,0.0,0.0,0.0,/workspace/data/categories/categories/unique.u...,664.0,61.0,0,663,user_shops
6,user_profile,"(Tags.USER, Tags.CATEGORICAL)",int64,False,False,None,0.0,0.0,0.0,/workspace/data/categories/categories/unique.u...,45.0,16.0,0,44,user_profile
7,user_group,"(Tags.USER, Tags.CATEGORICAL)",int64,False,False,None,0.0,0.0,0.0,/workspace/data/categories/categories/unique.u...,11.0,16.0,0,10,user_group
8,user_gender,"(Tags.USER, Tags.CATEGORICAL)",int64,False,False,None,0.0,0.0,0.0,/workspace/data/categories/categories/unique.u...,3.0,16.0,0,2,user_gender
9,user_age,"(Tags.USER, Tags.CATEGORICAL)",int64,False,False,None,0.0,0.0,0.0,/workspace/data/categories/categories/unique.u...,7.0,16.0,0,6,user_age


We expect the label names to be empty.

In [15]:
label_names = schema.select_by_tag(Tags.TARGET).column_names
label_names

[]

### Negative sampling

Many datasets for recommender systems contain implicit feedback with logs of user interactions like clicks, add-to-cart, purchases, music listening events, rather than explicit ratings that reflects user preferences over items. To be able to learn from implicit feedback, we use the general (and naive) assumption that the interacted items are more relevant for the user than the non-interacted ones.
In Merlin Models we provide some scalable negative sampling algorithms for the Item Retrieval Task. In particular, in this example, we use the `in-batch` sampling algorithm which uses the items interacted by other users as negatives within the same mini-batch.

### Building the Model

Now, let's build our Two-Tower model. In a nutshell, we aggregate all user features to feed in user tower and feed the item features to the item tower. Then we compute the positive score by multiplying the user embedding with the item embedding and sample negative items (read more about negative sampling [here](https://openreview.net/pdf?id=824xC-SgWgU) and [here](https://medium.com/mlearning-ai/overview-negative-sampling-on-recommendation-systems-230a051c6cd7)), whose item embeddings are also multiplied by the user embedding. Then we apply the loss function on top of the positive and negative scores.

We make sure that the mlp blocks used for the user and query towers have the same last dimension. This is needed because we will compute the dot product between the two towers' outputs to get the similarity scores.

In [16]:
tower_dim = 64 

# create user schema using USER tag
user_schema = schema.select_by_tag(Tags.USER)
# create user (query) tower input block
user_inputs = mm.InputBlockV2(user_schema)
# create user (query) encoder block
query = mm.Encoder(user_inputs, mm.MLPBlock([128, tower_dim], no_activation_last_layer=True))

# create item schema using ITEM tag
item_schema = schema.select_by_tag(Tags.ITEM)
# create item (candidate) tower input block
item_inputs = mm.InputBlockV2(item_schema)
# create item (candidate) encoder block
candidate = mm.Encoder(item_inputs, mm.MLPBlock([128, tower_dim], no_activation_last_layer=True))

`no_activation_last_layer:` when set True, no activation is used for top hidden layer. Learn more [here](https://storage.googleapis.com/pub-tools-public-publication-data/pdf/b9f4e78a8830fe5afcf2f0452862fb3c0d6584ea.pdf).

Build the model class.

In [17]:
model = mm.TwoTowerModelV2(query, candidate)

Note that in the `TwoTowerModelV2` function we did not set `negative_samplers` arg, that means it is set to None. In that case, Two-tower model is trained with contrastive learning and `in-batch` negative sampling strategy.

**Metrics:**

The following information retrieval metrics are used to compute the Top-10 accuracy of recommendation lists containing all items:

- **Normalized Discounted Cumulative Gain (NDCG@10)**: NDCG accounts for rank of the relevant item in the recommendation list and is a more fine-grained metric than HR, which only verifies whether the relevant item is among the top-k items.

- **Recall@10**: Also known as HitRate@n when there is only one relevant item in the recommendation list. Recall just verifies whether the relevant item is among the top-n items.

We need to initialize the dataloaders.

In [18]:
model.compile(optimizer="adam", run_eagerly=False, metrics=[mm.RecallAt(10), mm.NDCGAt(10)])
model.fit(train, validation_data=valid, batch_size=4096, epochs=2)

Epoch 1/2
86/86 [==============================] - 12s 48ms/step - loss: 8.2953 - recall_at_10: 0.0253 - ndcg_at_10: 0.0240 - regularization_loss: 0.0000e+00 - loss_batch: 8.2819 - val_loss: 8.2921 - val_recall_at_10: 0.0307 - val_ndcg_at_10: 0.0307 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 7.8969
Epoch 2/2
86/86 [==============================] - 3s 34ms/step - loss: 8.2949 - recall_at_10: 0.0303 - ndcg_at_10: 0.0302 - regularization_loss: 0.0000e+00 - loss_batch: 8.2815 - val_loss: 8.2924 - val_recall_at_10: 0.0225 - val_ndcg_at_10: 0.0206 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 7.8981


The validation metric values are calculated given the positive and negative scores in each batch, and then averaged over batches per epoch. That means validation metrics are not computed using the entire item catalog.

## Generate top-K recommendations

We trained a model, now we can generate recommendations offline using `to_top_k_encoder` method. The `to_top_k_encoder()` is a method of the RetrievalModelV2 class that uses the pre-trained candidate and query encoders to initialize a top-k encoder model, called the `topk_model` in this example. Practically, this method applies the candidate_encoder on the provided candidate_features dataset to set the top-k index of the `topk_model`. Therefore, topk_model object is the one responsible of generating the top-k predictions.

In [19]:
# Top-K evaluation
candidate_features = unique_rows_by_features(train, Tags.ITEM, Tags.ITEM_ID)
topk_model = model.to_top_k_encoder(candidate_features, k=20, batch_size=128)

# we can set `metrics` param in the `compile(), if we want
topk_model.compile(run_eagerly=False)

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


Note that above when we  set `validation_data=valid` in the model.fit(), we compute evaluation metrics on validation set using the negative sampling strategy used for training. Below, by using the `topk_model` we can evaluate the trained retrieval model using the entire catalog. This is doing dot product for entire catalog, and by default it is brute force.

In [20]:
loader = mm.Loader(train, batch_size=1024, transform=mm.ToTarget(schema, "item_id"))

metrics = topk_model.evaluate(loader, return_dict=True)
metrics

342/342 [==============================] - 7s 16ms/step - loss: 0.4353 - recall_at_10: 0.0987 - mrr_at_10: 0.0381 - ndcg_at_10: 0.0523 - map_at_10: 0.0381 - precision_at_10: 0.0099 - regularization_loss: 0.0000e+00 - loss_batch: 0.4353


{'loss': 0.4352704584598541,
 'recall_at_10': 0.09865741431713104,
 'mrr_at_10': 0.038072582334280014,
 'ndcg_at_10': 0.05225344002246857,
 'map_at_10': 0.038072582334280014,
 'precision_at_10': 0.00986573938280344,
 'regularization_loss': 0.0,
 'loss_batch': 0.45253440737724304}

Let's generate top-K (k=20 in our example) recommendations for a given batch of 8 samples. The `to_top_k_encoder()` method uses the index provided in the item (candidate) features dataset as the identifiers. The forward method of `topk_model` takes as the query features as input, and computes the dot product scores between the given query embeddings and all the candidates of the top-k index. Then, it returns the top-k (k=20) item ids with the highest scores. The output of the method is a named tuple `TopKPrediction`, where the first element is the dot product scores and the second element is the encoded item ids (not the original ids).

In [21]:
loader = mm.Loader(valid, batch_size=8, shuffle=False)
batch =next(iter(loader))

Let's check the `user_id` column in a given batch.

In [22]:
batch[0]['user_id']

<tf.Tensor: shape=(8, 1), dtype=int64, numpy=
array([[54],
       [33],
       [46],
       [17],
       [ 2],
       [13],
       [45],
       [ 6]])>

The recommended top 20 item ids are returned below.

In [23]:
topk_model(batch[0])

TopKPrediction(scores=<tf.Tensor: shape=(8, 20), dtype=float32, numpy=
array([[0.04311734, 0.03331283, 0.03179005, 0.03030831, 0.02860665,
        0.02391566, 0.022619  , 0.02032381, 0.01994756, 0.01979393,
        0.01941435, 0.01884781, 0.01881796, 0.01829395, 0.01812337,
        0.0172695 , 0.01512221, 0.01476259, 0.0144832 , 0.01414051],
       [0.06486996, 0.06063892, 0.05383713, 0.05328075, 0.05159207,
        0.05023589, 0.0495996 , 0.04927944, 0.04905936, 0.04812177,
        0.0475877 , 0.04532272, 0.04288622, 0.04199533, 0.04149817,
        0.04040359, 0.0402691 , 0.0401251 , 0.03897774, 0.03877449],
       [0.07392552, 0.03821031, 0.03790041, 0.03649426, 0.03353362,
        0.03333894, 0.02991731, 0.02968454, 0.02873009, 0.02823401,
        0.02703691, 0.02690915, 0.02670371, 0.02471804, 0.0238557 ,
        0.02352481, 0.02219179, 0.02209403, 0.02101746, 0.01856286],
       [0.02921886, 0.02902791, 0.02842918, 0.02763111, 0.02695863,
        0.02537199, 0.02488535, 0.02445289

## Exporting Retrieval Models

So far we have trained and evaluated our Retrieval model. Now, the next step is to deploy our model and generate top-K recommendations given a user (query). We can efficiently serve our model by indexing the trained item embeddings into an **Approximate Nearest Neighbors (ANN)** engine. Basically, for a given user query vector, that is generated passing the user features into user tower of retrieval model, we do an ANN search query to find the ids of nearby item vectors, and at serve time, we score user embeddings over all indexed top-K item embeddings within the ANN engine.

In doing so, we need to export
 
- user (query) tower
- item and user features
- item embeddings

#### Save and Load User (query) tower

We are able to save the user tower model as a TF model to disk. The user tower model is needed to generate a user embedding vector when a user feature vector <i>x</i> is fed into that model.

In [24]:
query_tower = model.query_encoder
query_tower.save(os.path.join(DATA_FOLDER, "query_tower"))

## we can load back the saved model via the following script.
#query_tower_loaded = tf.keras.models.load_model(os.path.join(DATA_FOLDER, 'query_tower'))

#### Extract and save User features

With `unique_rows_by_features` utility function we can easily extract both unique user and item features tables as cuDF dataframes. Note that for user features table, we use `USER` and `USER_ID` tags.

In [26]:
user_features = (
    unique_rows_by_features(train, Tags.USER, Tags.USER_ID).compute().reset_index(drop=True)
)

In [27]:
user_features.head()

,user_id,user_shops,user_profile,user_group,user_gender,user_age,user_consumption_2,user_is_occupied,user_geography,user_intentions,user_brands,user_categories
0,1,1,1,1,1,1,1,1,1,1,1,1
1,2,2,1,1,1,1,1,1,1,2,2,2
2,3,3,1,1,1,1,1,1,1,3,3,3
3,4,4,1,1,1,1,1,1,1,4,4,4
4,5,5,1,1,1,1,1,1,1,5,5,5


In [28]:
user_features.shape

(670, 12)

In [29]:
# save to disk
user_features.to_parquet(os.path.join(DATA_FOLDER, "user_features.parquet"))

#### Generate Query embeddings for entire user catalog

In [30]:
queries = model.query_embeddings(Dataset(user_features, schema=schema), batch_size=1024, index=Tags.USER_ID)
query_embs_df = queries.compute(scheduler="synchronous").reset_index()

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(


In [31]:
query_embs_df.head()

,user_id,0,1,2,3,4,5,6,7,8,...,54,55,56,57,58,59,60,61,62,63
0,1,0.007236,-0.011091,0.031452,0.036042,-0.026632,-0.036503,-0.014497,0.030610,-0.070837,...,-0.059863,-0.062461,-0.073296,0.017011,-0.035493,-0.032378,0.038639,0.049368,0.090955,-0.040021
1,2,-0.058328,0.002314,-0.025751,0.120739,-0.063952,-0.036477,-0.019444,0.012032,-0.059189,...,-0.044379,-0.085005,-0.092929,0.055135,-0.004949,-0.041250,-0.028414,0.042066,0.053964,-0.059259
2,3,-0.070867,0.012901,-0.091407,0.106760,-0.045025,-0.113616,0.061266,-0.037255,-0.012828,...,-0.119775,-0.036340,-0.117950,-0.035277,0.044963,0.000867,-0.077500,0.061420,0.005309,-0.056231
3,4,-0.086403,0.028190,-0.045930,0.124974,-0.084398,-0.103136,-0.090093,0.072658,-0.076329,...,-0.024317,-0.012980,-0.039977,0.043651,-0.037770,0.016203,-0.063164,-0.038701,0.015971,-0.062238
4,5,-0.078594,-0.008169,0.050156,0.104960,-0.080393,0.039646,-0.074536,0.032025,-0.069479,...,-0.026055,0.003718,-0.033900,0.020597,-0.030418,0.019828,-0.022337,0.011041,0.086443,-0.098106


#### Extract and save Item features

In [32]:
item_features = (
    unique_rows_by_features(train, Tags.ITEM, Tags.ITEM_ID).compute().reset_index(drop=True)
)

In [33]:
item_features.head()

,item_id,item_category,item_shop,item_brand
0,1,1,1,1
1,2,2,2,2
2,3,3,3,3
3,4,4,4,4
4,5,5,5,5


In [34]:
# save to disk
item_features.to_parquet(os.path.join(DATA_FOLDER, "item_features.parquet"))

#### Extract and save Item embeddings

In [35]:
item_embs = model.candidate_embeddings(Dataset(item_features, schema=schema), batch_size=1024, index=Tags.ITEM_ID)

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


In [36]:
item_embs_df = item_embs.compute(scheduler="synchronous")

In [37]:
item_embs_df

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
item_id,,,,,,,,,,,,,,,,,,,,,
1,0.069517,-0.104214,-0.075204,0.028716,-0.097462,-0.072198,0.019826,0.107861,-0.031005,-0.016088,...,-0.012549,0.043469,-0.009982,-0.065651,0.080480,0.009935,0.019771,0.014189,-0.027317,0.089973
2,-0.008808,0.025472,-0.016366,0.031776,0.015740,0.004447,0.067143,0.019933,0.019715,-0.084745,...,-0.063557,-0.049028,-0.020455,-0.050621,0.075051,-0.035535,-0.021190,0.061010,0.030041,0.098613
3,-0.014674,0.021853,-0.038137,0.070924,-0.027926,0.004813,-0.030839,0.077789,0.002299,0.039477,...,-0.002441,-0.003911,-0.045193,0.010367,0.020581,-0.002212,-0.045873,0.017771,-0.015072,0.049990
4,0.002860,0.028556,0.020693,0.037079,-0.019777,0.031904,-0.023743,0.041499,0.010521,0.024719,...,-0.034833,0.009799,-0.062456,0.000583,0.024885,-0.056013,-0.013155,0.003413,0.016525,0.066441
5,-0.035514,0.069924,-0.012289,0.056976,-0.041162,-0.077516,-0.053215,0.071526,-0.029758,0.061776,...,0.011410,0.021131,0.007661,0.058606,-0.076627,-0.029217,-0.006974,-0.034651,-0.072621,-0.006767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
658,0.015395,0.002248,0.060358,0.021226,-0.023938,-0.014744,0.029260,0.045533,-0.019517,-0.034832,...,0.008409,0.025469,0.054268,0.030795,-0.073625,-0.035433,0.042152,0.000461,-0.061142,0.047165
659,-0.001080,-0.061183,0.039221,-0.003353,0.006636,0.018814,-0.010222,0.044349,-0.036747,0.000195,...,-0.019220,-0.025662,-0.011337,-0.002444,-0.013816,-0.036031,0.006682,0.016636,-0.038625,0.005651
660,0.001210,-0.043627,0.020973,-0.000975,-0.012985,-0.060766,0.032327,0.055184,0.011252,-0.051540,...,-0.010178,-0.001657,0.012095,-0.028837,-0.030117,-0.001157,0.017854,0.000087,-0.037736,0.038003


In [38]:
# save to disk
item_embs_df.to_parquet(os.path.join(DATA_FOLDER, "item_embeddings.parquet"))

That's it. You have learned how to train and evaluate your Two-Tower retrieval model, and then how to export the required components to be able to deploy this model to generate recommendations. In order to learn more on serving a model to [Triton Inference Server](https://github.com/triton-inference-server/server), please explore the examples in the [Merlin](https://github.com/NVIDIA-Merlin/Merlin) and [Merlin Systems](https://github.com/NVIDIA-Merlin/systems) repos.